In [8]:
import finn
from finn.util.visualization import showSrc, showInNetron
from finn.util.basic import make_build_dir
import os
import onnx
file_path = str('../workspace/jh_fpga_amr/src/py/models')
file_name = str('/vgglike_5f_5c_4re_4mp_pr0_quant8.onnx')
onnx_model = onnx.load(file_path + file_name)
showInNetron(file_path + file_name)

Stopping http://0.0.0.0:8081
Serving '../workspace/jh_fpga_amr/src/py/models/vgglike_5f_5c_4re_4mp_pr0_quant8.onnx' at http://0.0.0.0:8081


In [9]:
from qonnx.util.cleanup import cleanup_model
from qonnx.core.modelwrapper import ModelWrapper
import os
os.makedirs(os.path.dirname(file_path + '/cleanup' + file_name), exist_ok=True)
export_onnx_path_cleaned = file_path + '/cleanup' + file_name
model = ModelWrapper(export_onnx_path_cleaned)
model = cleanup_model(model)
model.save(export_onnx_path_cleaned)
showInNetron(export_onnx_path_cleaned)

Stopping http://0.0.0.0:8081
Serving '../workspace/jh_fpga_amr/src/py/models/cleanup/vgglike_5f_5c_4re_4mp_pr0_quant8.onnx' at http://0.0.0.0:8081


In [10]:
from finn.transformation.qonnx.convert_qonnx_to_finn import ConvertQONNXtoFINN
from qonnx.core.modelwrapper import ModelWrapper
model_for_sim = ModelWrapper(export_onnx_path_cleaned)

from qonnx.core.datatype import DataType

finnonnx_in_tensor_name = model_for_sim.graph.input[0].name
finnonnx_out_tensor_name = model_for_sim.graph.output[0].name
print("Input tensor name: %s" % finnonnx_in_tensor_name)
print("Output tensor name: %s" % finnonnx_out_tensor_name)
finnonnx_model_in_shape = model_for_sim.get_tensor_shape(finnonnx_in_tensor_name)
finnonnx_model_out_shape = model_for_sim.get_tensor_shape(finnonnx_out_tensor_name)
print("Input tensor shape: %s" % str(finnonnx_model_in_shape))
print("Output tensor shape: %s" % str(finnonnx_model_out_shape))
finnonnx_model_in_dt = model_for_sim.get_tensor_datatype(finnonnx_in_tensor_name)
finnonnx_model_out_dt = model_for_sim.get_tensor_datatype(finnonnx_out_tensor_name)
print("Input tensor datatype: %s" % str(finnonnx_model_in_dt.name))
print("Output tensor datatype: %s" % str(finnonnx_model_out_dt.name))
print("List of node operator types in the graph: ")
print([x.op_type for x in model_for_sim.graph.node])

model = model_for_sim.transform(ConvertQONNXtoFINN())

export_onnx_path_converted = file_path + '/conv' + file_name
os.makedirs(os.path.dirname(export_onnx_path_converted), exist_ok=True)
model.save(export_onnx_path_converted)
showInNetron(export_onnx_path_converted)

Input tensor name: global_in
Output tensor name: global_out
Input tensor shape: [1, 2, 32]
Output tensor shape: [1, 9]
Input tensor datatype: FLOAT32
Output tensor datatype: FLOAT32
List of node operator types in the graph: 
['Quant', 'Quant', 'Quant', 'Quant', 'Quant', 'Quant', 'Quant', 'Quant', 'Quant', 'Quant', 'Quant', 'Conv', 'Relu', 'MaxPool', 'Conv', 'Relu', 'MaxPool', 'Conv', 'Relu', 'MaxPool', 'Conv', 'Relu', 'MaxPool', 'Conv', 'Relu', 'Reshape', 'Gemm', 'Relu', 'Gemm', 'Relu', 'Gemm', 'Relu', 'Gemm', 'Relu', 'Gemm', 'Relu', 'Quant']


/home/rothej/finn/deps/qonnx/src/qonnx/transformation/gemm_to_matmul.py:57: UserWarning: The GemmToMatMul transformation only offers explicit support for version 9 of the Gemm node, but the ONNX version of the supplied model is 11. Thus the transformation may fail or return incomplete results.
  warnings.warn(


Stopping http://0.0.0.0:8081
Serving '../workspace/jh_fpga_amr/src/py/models/conv/vgglike_5f_5c_4re_4mp_pr0_quant8.onnx' at http://0.0.0.0:8081


In [15]:
from qonnx.transformation.general import GiveReadableTensorNames, GiveUniqueNodeNames, RemoveStaticGraphInputs
from qonnx.transformation.infer_shapes import InferShapes
from qonnx.transformation.infer_datatypes import InferDataTypes
from qonnx.transformation.fold_constants import FoldConstants
from qonnx.transformation.change_3d_tensors_to_4d import Change3DTo4DTensors # New(ish) custom build op to handle Quant1D layers.
import finn.transformation.streamline.absorb as absorb
import finn.transformation.fpgadataflow.convert_to_hw_layers as to_hw
from finn.transformation.qonnx.qonnx_activation_handlers import QuantReluHandler

from qonnx.transformation.base import Transformation

class ApplyQuantReluHandler(Transformation):
    """ Custom transformation wrapper for the QuantReluHandler. """
    def apply(self, model):
        for node_index, node in enumerate(model.graph.node):
            if node.op_type == "Relu": 
                QuantReluHandler(model, node, node_index).transform(model)
                print("Debug")
        return (model, False)

model = model.transform(InferShapes())
model = model.transform(FoldConstants())
model = model.transform(GiveUniqueNodeNames())
model = model.transform(GiveReadableTensorNames())
model = model.transform(InferDataTypes())
model = model.transform(RemoveStaticGraphInputs())
model = model.transform(ApplyQuantReluHandler())
model = model.transform(Change3DTo4DTensors())
model = model.transform(absorb.AbsorbScalarMulAddIntoTopK())
model = model.transform(to_hw.InferChannelwiseLinearLayer())
model = model.transform(to_hw.InferLabelSelectLayer())
model = model.transform(GiveUniqueNodeNames())

os.makedirs(os.path.dirname(file_path + '/verif' + file_name), exist_ok=True)
verif_model_filename = file_path + '/verif' + file_name
model.save(verif_model_filename)
showInNetron(verif_model_filename)

AttributeError: 'QuantReluHandler' object has no attribute 'transform'

In [5]:
import finn.builder.build_dataflow as build
import finn.builder.build_dataflow_config as build_cfg
import os
import shutil

model_file = verif_model_filename

estimates_output_dir = "output_estimates_only"

# Delete previous run results if exist.
if os.path.exists(estimates_output_dir):
    shutil.rmtree(estimates_output_dir)
    print("Previous run results deleted!")


cfg_estimates = build.DataflowBuildConfig(
    output_dir          = estimates_output_dir,
    mvau_wwidth_max     = 80,
    target_fps          = 1000000,
    synth_clk_period_ns = 10.0,
    fpga_part           = "xc7z020clg400-1",
    steps               = build_cfg.estimate_only_dataflow_steps,
    generate_outputs=[
        build_cfg.DataflowOutputType.ESTIMATE_REPORTS,
    ]
)

Previous run results deleted!


In [6]:
%%time
build.build_dataflow_cfg(model_file, cfg_estimates);

Building dataflow accelerator from ../workspace/jh_fpga_amr/src/py/models/verif/vgglike_5f_5c_4re_4mp_pr0_quant8.onnx
Intermediate outputs will be generated in /tmp/finn_dev_rothej
Final outputs will be generated in output_estimates_only
Build log is at output_estimates_only/build_dataflow.log
Running step: step_qonnx_to_finn [1/10]
Running step: step_tidy_up [2/10]
Running step: step_streamline [3/10]


Traceback (most recent call last):
  File "/home/rothej/finn/src/finn/builder/build_dataflow.py", line 158, in build_dataflow_cfg
    model = transform_step(model, cfg)
  File "/home/rothej/finn/src/finn/builder/build_dataflow_steps.py", line 317, in step_streamline
    model = model.transform(LowerConvsToMatMul())
  File "/home/rothej/finn/deps/qonnx/src/qonnx/core/modelwrapper.py", line 140, in transform
    (transformed_model, model_was_changed) = transformation.apply(transformed_model)
  File "/home/rothej/finn/deps/qonnx/src/qonnx/transformation/lower_convs_to_matmul.py", line 77, in apply
    k_w = k[1]
IndexError: list index (1) out of range


> /home/rothej/finn/deps/qonnx/src/qonnx/transformation/lower_convs_to_matmul.py(77)apply()
     75                 k = get_by_name(n.attribute, "kernel_shape").ints
     76                 k_h = k[0]
---> 77                 k_w = k[1]
     78                 stride_h = get_by_name(n.attribute, "strides").ints[0]
     79                 stride_w = get_by_name(n.attribute, "strides").ints[1]

--KeyboardInterrupt--

KeyboardInterrupt: Interrupted by user
Build failed
CPU times: user 2.88 s, sys: 408 ms, total: 3.29 s
Wall time: 2min 20s


In [7]:
assert os.path.exists(estimates_output_dir + "/report/estimate_layer_resources.json")

AssertionError: 